In [131]:
# For Define file path
import os

import shutil
import time
from threading import Timer
from time import gmtime, strftime

# for tranform image to data
import numpy as np
import pandas as pd

# for plat image
from matplotlib import pyplot
import matplotlib.pyplot as plt
from matplotlib.patches import Patch

from skimage import measure

import rasterio
from rasterio.transform import from_origin
from rasterio.enums import Resampling
import geopandas as gpd
from rasterio.features import shapes
from shapely.geometry import shape



import warnings
import csv
warnings.filterwarnings(action='ignore') 

In [132]:
# Global variables:
systemCooldown = 2
Error_Limit = 1
mode = True

In [133]:
# File Paths
Drive = "/Users/imdsk/RIDAProjectBurnScar/Sentinel_Process/"
Image = os.path.join(Drive, "Image")
Image_Pre = os.path.join(Drive, "Image_Pre/")
Image_Finish = os.path.join(Drive, "Image_Finish/")
Image_Missing = os.path.join(Drive, "Image_Missing/")
Output = os.path.join(Drive, "Output/")
Rtbcon = os.path.join(Drive, "Raster_BurnCon/")
Rtbreg = os.path.join(Drive, "Raster_BurnReg/")
RtbShape = os.path.join(Drive, "Raster_BurnShape/")
RtbLevel = os.path.join(Drive, "Raster_BurnLevel/")
Track_arr = ["T4/", "T104/", "T61/", "T18/", "T118/"]

Function to load cooldown

In [134]:
# Function to load cooldown
def loadCooldown():
    global mode

Function to print time

In [135]:
# Function to print time
def print_time():
    return time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())

Function to Move File

In [136]:
# Function to move file
def Move_File(FileName, CurrDir, DestDir):
    try:
        if os.path.exists(os.path.join(CurrDir, FileName)):
            if os.path.exists(os.path.join(DestDir, FileName)):
                os.remove(os.path.join(DestDir, FileName))
            shutil.copy(os.path.join(CurrDir, FileName), DestDir)
            t = Timer(1, loadCooldown)
            t.start()
            t.join()
            os.remove(os.path.join(CurrDir, FileName))
            print(print_time() + f"Raster_Process :: Move File {FileName} Complete")
    except Exception as e:
        print(print_time() + f"Raster_Process :: Can not Move File {FileName}")
        print(print_time() + str(e))

Functuin to Write CSV

In [137]:
# Function to Write CSV
def write_to_csv(data, file_name_csv):
    try:
        # Convert NumPy array to a nested Python list or dictionary
        if isinstance(data, np.ndarray):
            data_list = data.tolist()  # Convert ndarray to a nested list
        else:
            data_list = data  # For other non-ndarray data, assume it's already in list format

        # Write to CSV file
        with open(file_name_csv, 'w', newline='') as csv_file:
            csv_writer = csv.writer(csv_file)
            for row in data_list:
                csv_writer.writerow(row)
        print(f"Data successfully written to {file_name_csv}")

    except Exception as e:
        print(f"Error writing to CSV file: {e}")

def save_results_to_csv(data_arrays, csv_filenames, window_size, track):
    """Saves generated data arrays into CSV files within a track-specific folder"""

    output_dir = f"output_data/{track}"  # Example output structure
    os.makedirs(output_dir, exist_ok=True)  # Ensure the output directory exists

    for data, filename in zip(data_arrays, csv_filenames):
        full_path = os.path.join(output_dir, filename)
        np.savetxt(full_path, data, delimiter=',')

# def save_results_to_csv(results, filenames, window_size):
#         """Saves calculated index data to CSV files."""

#         for data, filename in zip(results, filenames):
#             df = pd.DataFrame(data)
#             # df.index = np.arange(1, window_size + 1) 
#             # df.columns = create_header(window_size)
#             df.to_csv(filename, index=False, header=False) 
                       

In [138]:
def print_with_tag(data, tag):
    print("------")
    print(tag)
    print(data)
    print("------")
    

# Function to Resample

```:
Example use:

input_raster = AFB12
output_raster = AFB1210
new_resolution = 10 

resample_raster(input_raster, output_raster, new_resolution)

```

In [139]:
# Function to Resample
def resample_raster(input_raster, output_raster, new_resolution):
                with rasterio.open(input_raster) as src:
                    transform = src.transform
                    data = src.read(
                        out_shape=(
                            src.count,
                            int(src.height * src.transform.a / new_resolution),
                            int(src.width * src.transform.a / new_resolution)
                        ),
                        resampling=Resampling.nearest  # You can adjust the resampling method here
                    )
                    profile = src.profile
                    profile.update(transform=transform, width=data.shape[2], height=data.shape[1])

                    with rasterio.open(output_raster, 'w', **profile) as dst:
                        dst.write(data)

# Function to Create Shape

```:
Example use:

input_raster = raster
output_path = Path/To/Destination + Short_name+ "_Example" + ".tif"
save_as_geotiff(input_raster, output_path)

```

In [140]:
# Function to Create Shape
def save_as_geotiff(data, output_path, resolution=10):
                    height, width = data.shape

                    # Define the transformation
                    transform = from_origin(0, height * resolution, resolution, -resolution)

                    # Define the profile
                    profile = {
                        'driver': 'GTiff',
                        'count': 1,
                        'dtype': 'uint8',
                        'width': width,
                        'height': height,
                        'crs': 'EPSG:4326',  # Adjust the CRS as needed
                        'transform': transform,
                        'compress': 'packbits',
                        'tiled': True,
                        'interleave': 'band',
                    }

                    with rasterio.open(output_path, 'w', **profile) as dst:
                        dst.write(data.astype('uint8'), 1)

# Crop Table

# Formula Calculate Indices
```:
BFB   Stand for Before burn
AFB   Stand for After burn
B02   Stand for Blue 
B03   Stand for Green 
B04   Stand for Red 
b08   Stand for VNIR (Visible Near Infared)
b12   Stand for SWIR (Shor Wave Infared)
b1210 Stand for SWIR Resample from 20m to 10m

nbr   Stand for Nomalize Burn Ratio
```

In [141]:

# def calculate_post_nbr(afb_B08,afb_B1210):
#                     post_nbr= (afb_B08 - afb_B1210) / (afb_B08 + afb_B1210) 
#                     return post_nbr

# def calculate_dnbr(pre_nbr,post_nbr):
#                     dnbr = pre_nbr - post_nbr        
#                     return dnbr

# def calculate_ndwi(afb_B03,afb_B08):             
#                     ndwi = (afb_B03 - afb_B08) / (afb_B03 + afb_B08) #(AFB03 - AFB08) /(AFB03 + AFB08)                
#                     return ndwi

# def calculate_ndvi(post_red,post_nir):                                 
#                     ndvi_data = (post_nir - post_red) / (post_nir + post_red) #(AFB08 - AFB04) / (AFB08 + AFB04)             
#                     return ndvi_data


# Condition to feature table



In [142]:
# def lv1(dnbr):  
#         burn_con_lv1 = np.where((dnbr > 0.27),1,0) 
#         return burn_con_lv1

# def lv2(ndwi):
#         burn_con_lv2 = np.where((ndwi < 0),2,0)
#         return burn_con_lv2

# def lv3(ndvi):        
#         burn_con_lv3 = np.where((ndvi > 0.14),3,0)
#         return burn_con_lv3
        

# def lv4(nir):
#         burn_con_lv4 = np.where((nir < 2500),4,0)
#         return burn_con_lv4 


        

# def burn_con(dNBR, data_AFB03,data_AFB04, data_AFB08, output_path):
                    
                    

#                     # burnCon_LV1 = np.where(((dNBR > 0.44) & (PostNBR_data < 0)), 1, 0)
#                     # burnCon_LV1 = np.where(((dNBR > 0.27) & (PostNBR_data < 0.10)),1,0) 
#                     # burnCon_LV1 = np.where((dNBR > 0.27),1,0) 
#                     burnCon_LV1 = np.where((dNBR > 0.27)&(PNDVI_data<0.14),1,0) 
#                     # write_to_csv(burnCon_LV1, '[CSV]burncon_lv1.csv')
#                     # print(burnCon_LV1)
#                     # print("After Lv1")

#                     burnCon_Final = np.where(((burnCon_LV1 == 1) & (data_AFB08 < 2500) & (data_AFB08 > data_AFB03)), 1, 0)
#                     # burnCon_Final = np.where(((burnCon_LV1 == 1) & (data_AFB08 < 2000) & (data_AFB08 > data_AFB03)), 1, 0)
#                     # write_to_csv(burnCon_Final, '[CSV]burnCon_Final.csv')
#                     # print(burnCon_Final)
#                     # print("After condition final")
                    

#                     # Save the result to a GeoTIFF file
#                     save_as_geotiff(burnCon_Final, output_path)

#                 # # Example usage
#                 # output_path = Rtbcon + Track + Full_name + ".tif"
#                 # burn_con(dNBR, data_AFB03,data_AFB04, data_AFB08, output_path)

# Normalize

In [143]:
# def normalize_minmax(dNBR):
#     dNBR_normalized = (dNBR - dNBR.min()) / (dNBR.max() - dNBR.min())
#     return dNBR_normalized


# Crop Image


# Down sampling

In [144]:
# Function to process raster

def Raster_Process(Track):
    global mode, Image, Image_Pre, Image_Finish, Image_Missing, Output, Error_Limit

    Loop_Limit = 0
    image_track = os.path.join(Image, Track)
    output_track = os.path.join(Output, Track)

    rasters = [r for r in os.listdir(image_track) if r.endswith('B12.jp2')]

    for raster in rasters:
        if Loop_Limit > 0:
            mode = True
            return

        Full_name = os.path.splitext(raster)[0]
        Full_name_parts = Full_name.split('_')
        Mid_name = Full_name[:23]
        Short_name = Full_name[:6]
        # Short_name = Full_name_parts[0] + '_' + Full_name_parts[1]
        # print(Full_name)
        # print(Short_name)

        BFB02 = os.path.join(Image_Pre, Track, f"{Short_name}_B02.jp2")
        BFB03 = os.path.join(Image_Pre, Track, f"{Short_name}_B03.jp2")
        BFB04 = os.path.join(Image_Pre, Track, f"{Short_name}_B04.jp2")
        BFB05 = os.path.join(Image_Pre, Track, f"{Short_name}_B05.jp2")
        BFB06 = os.path.join(Image_Pre, Track, f"{Short_name}_B06.jp2")
        BFB07 = os.path.join(Image_Pre, Track, f"{Short_name}_B07.jp2")
        BFB08 = os.path.join(Image_Pre, Track, f"{Short_name}_B08.jp2")
        BFB8A = os.path.join(Image_Pre, Track, f"{Short_name}_B8A.jp2")
        BFB09 = os.path.join(Image_Pre, Track, f"{Short_name}_B09.jp2")
        BFB12 = os.path.join(Image_Pre, Track, f"{Short_name}_B12.jp2")
        BFB1210 = os.path.join(Image_Pre, Track, f"{Short_name}_B1210.jp2")
        BFNBR = os.path.join(Image_Pre, Track, f"{Short_name}_NBR.tif")

        AFB02 = os.path.join(Image, Track, f"{Mid_name}B02.jp2")
        AFB03 = os.path.join(Image, Track, f"{Mid_name}B03.jp2")
        AFB04 = os.path.join(Image, Track, f"{Mid_name}B04.jp2")
        AFB05 = os.path.join(Image, Track, f"{Mid_name}B05.jp2")
        AFB06 = os.path.join(Image, Track, f"{Mid_name}B06.jp2")
        AFB07 = os.path.join(Image, Track, f"{Mid_name}B07.jp2")
        AFB08 = os.path.join(Image, Track, f"{Mid_name}B08.jp2")
        AFB8A = os.path.join(Image, Track, f"{Mid_name}B8A.jp2")
        AFB09 = os.path.join(Image, Track, f"{Mid_name}B09.jp2")
        AFB12 = os.path.join(Image, Track, f"{Mid_name}B12.jp2")
        AFB1210 = os.path.join(Image, Track, f"{Mid_name}B1210.jp2")

        print(BFB12)
       

        if all(
            [ os.path.exists(f) for f in [BFB02,BFB03,BFB04,BFB05,BFB06,BFB07,BFB08,BFB8A,BFB09,BFB1210,AFB02,AFB03,AFB04,AFB05,AFB06,AFB07,AFB08,AFB8A,AFB09,AFB1210]
                
            ]
        ):
            print(print_time() + "Raster_Process :: Start GIS Process Please Wait....")
            Loop_Limit += 1
            # Delete Old File
            print(print_time() + "Raster_Process :: Delete Old File")
            files = os.listdir(output_track)
            for f in files:
                try:
                    os.remove(os.path.join(output_track, f))
                except Exception as e:
                    print(print_time() + f"Raster_Process :: Cannot Delete {f}")
                    print(print_time() + str(e))
                    break

            print(print_time() + "Raster_Process :: Delete Old File Complete")
            t = Timer(3, loadCooldown)
            t.start()
            t.join()

            def resample_raster(input_raster, output_raster, new_resolution):
                with rasterio.open(input_raster) as src:
                    transform = src.transform
                    data = src.read(
                        out_shape=(
                            src.count,
                            int(src.height * src.transform.a / new_resolution),
                            int(src.width * src.transform.a / new_resolution)
                        ),
                        resampling=Resampling.nearest  # You can adjust the resampling method here
                    )
                    profile = src.profile
                    profile.update(transform=transform, width=data.shape[2], height=data.shape[1])

                    with rasterio.open(output_raster, 'w', **profile) as dst:
                        dst.write(data)
                            

            def save_as_geotiff(data, output_path, resolution=10):
                    height, width = data.shape

                    # Define the transformation
                    transform = from_origin(0, height * resolution, resolution, -resolution)

                    # Define the profile
                    profile = {
                        'driver': 'GTiff',
                        'count': 1,
                        'dtype': 'uint8',
                        'width': width,
                        'height': height,
                        'crs': 'EPSG:4326',  # Adjust the CRS as needed
                        'transform': transform,
                        'compress': 'packbits',
                        'tiled': True,
                        'interleave': 'band',
                    }

                    with rasterio.open(output_path, 'w', **profile) as dst:
                        dst.write(data.astype('uint8'), 1)

            if os.path.exists(BFB1210) == False:
                print(print_time() + "Raster_Process :: Resample " + Short_name)

                    # Usage:
                input_raster = BFB12
                output_raster = BFB1210
                new_resolution = 10  # Change this to the desired resolution

                resample_raster(input_raster, output_raster, new_resolution)


            if os.path.exists(BFNBR) == False:
                print(print_time()+"Raster_Process :: NBR " + Short_name)
                with rasterio.open(BFB08) as src_BFB08, rasterio.open(BFB1210) as src_BFB1210:
                    data_BFB08 = src_BFB08.read(1)  # Read the data of BFB08 raster band 1
                    data_BFB1210 = src_BFB1210.read(1)  # Read the data of BFB1210 raster band 1

                    NBR_data = (data_BFB08 - data_BFB1210) / (data_BFB08 + data_BFB1210)

                    profile = src_BFB08.profile  # Use BFB08 profile as a template for the new raster
                    output_path_NBR = Image_Pre + Track + Short_name + "_NBR.tif"
                    with rasterio.open(output_path_NBR, 'w', **profile) as dst:
                        dst.write(NBR_data, 1)


            if os.path.exists(AFB1210) == False:
                print(print_time()+"Raster_Process :: Resample " + Mid_name)

                # Usage:
                input_raster = AFB12
                output_raster = AFB1210
                new_resolution = 10 

                resample_raster(input_raster, output_raster, new_resolution)
            
            def normalize_dnbr(dnbr):
                """
                Normalizes dnbr values to a range between 0 and 1.

                Args:
                    dnbr: A numpy array of dnbr values.

                Returns:
                    A numpy array of normalized dnbr values.
                """

                # Handle negative values
                if np.min(dnbr) < 0:
                    dnbr = dnbr + np.abs(np.min(dnbr))

                # Normalize to range [0, 1]
                return (dnbr - np.min(dnbr)) / (np.max(dnbr) - np.min(dnbr))
    

            try:
                ########################################## Raster PROCESS ################################################
                print(print_time()+"Raster_Process :: Raster Process " + Full_name[:22])

                # Parameters
                rows_per_file = 549
                cols_per_file = 549
                output_file_index = 0 

                # Track processed count to match 10980x10980 dimensions
                total_processed = 0
                max_processed = 10980 * 10980


                data = {
                'PreNBR_data': [],
                'PostNBR_data': [],
                'dNBR': [],
                'NDVI':[],
                'NDWI':[],
                'data_AFB03': [],
                'data_AFB04': [],
                'data_AFB05': [],
                'data_AFB06': [],
                'data_AFB07': [],
                'data_AFB08': [],
                'data_AFB8A': [],
                'data_AFB09': [],
                'data_AFB1210': [],
                'Label': []
                }

                with rasterio.open(BFNBR) as src_BFNBR:
                    data_BFNBR = src_BFNBR.read(1)

                    PreNBR_data = (data_BFNBR)
                    print_with_tag(PreNBR_data, 'PreNBR_data')

                    # Append to CSV data
                    data['PreNBR_data'].extend(PreNBR_data.ravel())


                with rasterio.open(BFB02) as src_BFB02,rasterio.open(AFB02) as src_AFB02:
                    data_BFB02 = src_BFB02.read(1)
                    data_AFB02 = src_AFB02.read(1) 
                
                with rasterio.open(BFB03) as src_BFB03,rasterio.open(AFB03) as src_AFB03:
                    data_BFB03 = src_BFB03.read(1) 
                    data_AFB03 = src_AFB03.read(1)   

                with rasterio.open(BFB04) as src_BFB04,rasterio.open(AFB04) as src_AFB04:
                    data_BFB04 = src_BFB04.read(1)
                    data_AFB04 = src_AFB04.read(1)  
                
                with rasterio.open(BFB05) as src_BFB05,rasterio.open(AFB05) as src_AFB05:
                    data_BFB05 = src_BFB05.read(1)
                    data_BFB05 = src_AFB05.read(1)
                
                with rasterio.open(BFB06) as src_BFB06,rasterio.open(AFB06) as src_AFB06:
                    data_BFB06 = src_BFB06.read(1)
                    data_AFB06 = src_AFB06.read(1)
                
                with rasterio.open(BFB07) as src_BFB07,rasterio.open(AFB07) as src_AFB07:
                    data_BFB07 = src_BFB07.read(1)
                    data_AFB07 = src_AFB07.read(1)
                
                with rasterio.open(BFB08) as src_BFB08, rasterio.open(BFB8A) as src_BFB8A,rasterio.open(AFB08) as src_AFB08, rasterio.open(AFB8A) as src_AFB8A:
                    data_BFB08 = src_BFB08.read(1)
                    data_BFB8A = src_BFB8A.read(1)
                    data_AFB08 = src_AFB08.read(1)
                    data_AFB8A = src_AFB8A.read(1)
                
                with rasterio.open(BFB09) as src_BFB09,rasterio.open(AFB09) as src_AFB09:
                    data_BFB09 = src_BFB09.read(1) 
                    data_AFB09 = src_AFB09.read(1) 
                
                with rasterio.open(BFB1210) as src_BFB1210,rasterio.open(AFB1210) as src_AFB1210:
                    data_BFB1210 = src_BFB1210.read(1) 
                    data_AFB1210 = src_AFB1210.read(1)

                 # Append to CSV data
                data['data_AFB03'].extend(data_AFB03.ravel())
                data['data_AFB04'].extend(data_AFB04.ravel())
                data['data_AFB05'].extend(data_AFB03.ravel())
                data['data_AFB06'].extend(data_AFB03.ravel())
                data['data_AFB07'].extend(data_AFB03.ravel())
                data['data_AFB08'].extend(data_AFB03.ravel())
                data['data_AFB8A'].extend(data_AFB08.ravel())
                data['data_AFB09'].extend(data_AFB03.ravel())
                data['data_AFB1210'].extend(data_AFB1210.ravel())


                print_with_tag(data_BFNBR,'data_BFNBR')
                print_with_tag(data_AFB02, 'data_AFB02')
                print_with_tag(data_AFB03, 'data_AFB03')
                print_with_tag(data_AFB08, 'data_AFB08')
                print_with_tag(data_AFB1210, 'data_AFB1210')


                PreNBR_data = (data_BFNBR)
                print_with_tag(PreNBR_data, 'PreNBR_data')
                PostNBR_data = (data_AFB08 - data_AFB1210) / (data_AFB08 + data_AFB1210)
                print_with_tag(PostNBR_data, 'PostNBR_data')

                PNDVI_data = (data_AFB08 - data_AFB04) / (data_AFB08 + data_AFB04)

                # Append to CSV data
                data['PreNBR_data'].extend(PreNBR_data.ravel())
                data['PostNBR_data'].extend(PostNBR_data.ravel())

                    
                    

                ########################################## Burn Condition ################################################
               
                print(print_time()+"Raster_Process :: Burn Raster Condition")

                
                dNBR = PreNBR_data - PostNBR_data
                dNDWI = (data_AFB03 - data_AFB08) /(data_AFB03 + data_AFB08)
                dNDVI = (data_AFB08-data_AFB04) / (data_AFB08+data_AFB04)


                ########################################## Lv1 ################################################
                
                def burn_con_lv1(dNBR, dNDWI, dNDVI, data_AFB03,data_AFB04, data_AFB08, output_path):

                     
                    burn_con_lv1 = np.where((dNBR > 0.27)&(PostNBR_data < 0.10),1,0) 
                  
                
                    save_as_geotiff(burn_con_lv1, output_path)

                # Example usage
                output_path = RtbLevel + Track + Full_name + "Lv1.tif" # save this as Lv 1 2 3 4 
                burn_con_lv1(dNBR, dNDWI, dNDVI, data_AFB03,data_AFB04, data_AFB08, output_path)


                ########################################## Lv2 ################################################

                def burn_con_lv2(dNBR, dNDWI, dNDVI, data_AFB03,data_AFB04, data_AFB08, output_path):

                     
                    burn_con_lv2 = np.where((data_AFB08 > data_AFB03),1,0)
                  
                
                    save_as_geotiff(burn_con_lv2, output_path)

                # Example usage
                output_path = RtbLevel + Track + Full_name + "Lv2.tif" # save this as Lv 1 2 3 4 
                burn_con_lv2(dNBR, dNDWI, dNDVI, data_AFB03,data_AFB04, data_AFB08, output_path)


                ########################################## Lv3 ################################################

                def burn_con_lv3(dNBR, dNDWI, dNDVI, data_AFB03,data_AFB04, data_AFB08, output_path):

                     
                    burn_con_lv3 = np.where((data_AFB08 < 2500),1,0)
                  
                
                    save_as_geotiff(burn_con_lv3, output_path)

                # Example usage
                output_path = RtbLevel + Track + Full_name + "Lv3.tif" # save this as Lv 1 2 3 4 
                burn_con_lv3(dNBR, dNDWI, dNDVI, data_AFB03,data_AFB04, data_AFB08, output_path)


                ########################################## Burn Condition ################################################

                def burn_con(dNBR, dNDWI, dNDVI, data_AFB03,data_AFB04, data_AFB08, output_path):

                     
                    burn_con_lv1 = np.where((dNBR > 0.27)&(PostNBR_data < 0.10),1,0) 
                    burn_con_lv2 = np.where((data_AFB08 > data_AFB03),1,0)
                    burn_con_lv4 = np.where((data_AFB08 < 2500),1,0)

                  
                    burnCon_Final = np.where(((burn_con_lv1 == 1) & (burn_con_lv2 == 1)&(burn_con_lv4== 1)), 1, 0)
                
                    save_as_geotiff(burnCon_Final, output_path)

                # Example usage
                output_path = Rtbcon + Track + Full_name + ".tif" # save this as Lv 1 2 3 4 
                burn_con(dNBR, dNDWI, dNDVI, data_AFB03,data_AFB04, data_AFB08, output_path)
                
                print(print_time()+"we got dataframe")
                ##########################################################################################################

                

                threshold_value = 20
                print(print_time()+"Raster_Process :: RegionGroup")

                BCON = os.path.join(Rtbcon, Track, f"{Full_name}.tif")
                
                with rasterio.open(BCON) as src_BCON:
                    data_BCON = src_BCON.read(1) 


                def burn_region(data_BCON, track, full_name):
                    # Perform BurnRegion operations here
                    burnRegionGrp = measure.label(data_BCON, connectivity=2)  # Equivalent to RegionGroup
                    print(print_time() + f"Raster_Process :: Region > {threshold_value}")

                    # Use numpy to create a mask where count > 400
                    burnRegionGrpThresholded = np.where(burnRegionGrp > threshold_value, 1, 0)



                    burnRegion = np.where(burnRegionGrpThresholded == 1, 1, 0)
                    # Append to CSV data
                    data['Label'].extend(burnRegion.ravel())

                    print(print_time() + "Raster_Process :: Save Raster")

                    output_path = Rtbreg + track + Short_name+ "_B12" + ".tif"
                    # output_path = Rtbreg + track + full_name+ ".tif"
                    save_as_geotiff(burnRegion, output_path)
                    
                    return burnRegion
                
                burn_region(data_BCON, Track, Full_name)

                # burnRegion = burn_region(data_BCON, Track, Full_name)

                # for row_start in range(0, 10980, rows_per_file):
                #     for col_start in range(0, 10980, cols_per_file):
                #         # Check if we've processed enough elements
                #         if total_processed >= max_processed:
                #             break

                #         # Subset of dNBR data (adjust if needed)
                #         dNBR_chunk = dNBR[row_start:row_start + rows_per_file, col_start:col_start + cols_per_file]

                #         # Normalize dNBR chunk
                #         normalized_dNBR = normalize_dnbr(dNBR_chunk) 
                        

                #         # Create a DataFrame for this chunk
                #         df_part = pd.DataFrame({
                #             'PreNBR_data': PreNBR_data[row_start:row_start + rows_per_file, col_start:col_start + cols_per_file].ravel(), 
                #             'PostNBR_data': PostNBR_data[row_start:row_start + rows_per_file, col_start:col_start + cols_per_file].ravel(), 
                #             'dNBR': normalized_dNBR.ravel(), 
                #             'data_AFB03': data_AFB03[row_start:row_start + rows_per_file, col_start:col_start + cols_per_file].ravel(), 
                #             'data_AFB04': data_AFB04[row_start:row_start + rows_per_file, col_start:col_start + cols_per_file].ravel(), 
                #             'data_AFB05': data_AFB04[row_start:row_start + rows_per_file, col_start:col_start + cols_per_file].ravel(), 
                #             'data_AFB06': data_AFB04[row_start:row_start + rows_per_file, col_start:col_start + cols_per_file].ravel(), 
                #             'data_AFB07': data_AFB04[row_start:row_start + rows_per_file, col_start:col_start + cols_per_file].ravel(), 
                #             'data_AFB08': data_AFB04[row_start:row_start + rows_per_file, col_start:col_start + cols_per_file].ravel(), 
                #             'data_AFB8A': data_AFB08[row_start:row_start + rows_per_file, col_start:col_start + cols_per_file].ravel(),
                #             'data_AFB09': data_AFB04[row_start:row_start + rows_per_file, col_start:col_start + cols_per_file].ravel(), 
                #             'data_AFB1210': data_AFB1210[row_start:row_start + rows_per_file, col_start:col_start + cols_per_file].ravel(),
                #             'Label': burnRegion[row_start:row_start + rows_per_file, col_start:col_start + cols_per_file].ravel()
                           
                #         })

                #         # Write this chunk to a CSV file
                #         output_filename = f"{Full_name}_burn_data_part_{output_file_index}.csv"
                #         output_path = os.path.join(Rtbcon + Track, output_filename)
                #         df_part.to_csv(output_path, index=False)

                #         total_processed += rows_per_file * cols_per_file
                #         output_file_index += 1
               

                print(print_time()+"Raster_Process :: Burn Raster Process Complate")

                ########################################## Vecter PROCESS #################################################
                def reclassify_raster(input_raster, output_raster, remap_dict):
                    with rasterio.open(input_raster) as src:
                        data = src.read(1)  # Assuming a single-band raster, adjust if necessary
                        profile = src.profile

                        # Reclassify using numpy's vectorized operations
                        for old_value, new_value in remap_dict.items():
                            data = np.where(data == old_value, new_value, data)

                        with rasterio.open(output_raster, 'w', **profile) as dst:
                            dst.write(data, 1)

                # Example usage:
                burnRegion = os.path.join(Rtbreg, Track, f"{Short_name}_B12.tif")
                burnReclass = os.path.join(Rtbreg, Track, f"{Short_name}_BurnReclass.tif")
                remap_dict = {1: 1}  # Add your reclassification rules here

                reclassify_raster(burnRegion, burnReclass, remap_dict)

                print("After burn reclasss")
                
            

                def raster_to_polygon(input_raster, output_shapefile, simplify=0, value_field="VALUE"):
                    with rasterio.open(input_raster) as src:
                        image = src.read(1)

                        # Get shapes and transform
                        shapes = rasterio.features.shapes(image, transform=src.transform)

                        # Create lists to collect geometries and values
                        geometries = []
                        values = []

                        for geom, value in shapes:
                            if simplify > 0:
                                geom = shape(geom).simplify(simplify)

                            geometries.append(geom)
                            values.append(value)

                        # Create GeoDataFrame
                        gdf = gpd.GeoDataFrame({value_field: values, 'geometry': geometries})

                        # Simplify each geometry in the GeoDataFrame
                        if simplify > 0:
                            gdf['geometry'] = gdf['geometry'].apply(lambda x: x.simplify(simplify))

                        # Dissolve overlapping polygons
                        dissolved_gdf = gdf.dissolve(by=value_field)

                        # Write to shapefile
                        dissolved_gdf.to_file(output_shapefile)

                # Example usage:
                burnReclass = os.path.join(Rtbreg, Track, f"{Short_name}_BurnReclass.tif")
                output_shapefile = RtbShape + Track + Short_name+ "_B12" + ".shp"
                simplify_value = 0.1  # Adjust this value if needed

                raster_to_polygon(burnReclass, output_shapefile, simplify=simplify_value)
                
                print(print_time()+"Raster_Process :: Start Move File ")
                Move_File(Mid_name + "B03.jp2", Image + Track, Image_Finish + Track)
                Move_File(Mid_name + "B04.jp2", Image + Track, Image_Finish + Track)
                Move_File(Mid_name + "B08.jp2", Image + Track, Image_Finish + Track)
                Move_File(Mid_name + "B12.jp2", Image + Track, Image_Finish + Track)
                Move_File(Mid_name + "B1210.jp2", Image + Track, Image_Finish + Track)
                Move_File(Mid_name + "B1210.jp2.aux.xml", Image + Track, Image_Finish + Track)
                Move_File(Mid_name + "B1210.jp2.ovr", Image + Track, Image_Finish + Track)
                Move_File(Mid_name + "B1210.jp2.xml", Image + Track, Image_Finish + Track)
                Error_Limit = 2
                print(print_time()+"Raster_Process :: Raster_Process ALL Complate >>>>>>>>>>>>>>>>>>>> \n \n")

            

                # Load the shapefile and image

                
                shapefile_path = os.path.join(RtbShape, Track, f"{Short_name}_B12.shp")
                image_path = os.path.join(Image, Track, f"{Full_name}10.jp2")

                gdf = gpd.read_file(shapefile_path)

                # Plot the image
                image = plt.imread(image_path)
                fig, ax = plt.subplots(figsize=(10, 10))
                ax.imshow(image, cmap='gray')

                # Plot the shapefile on top of the image
                gdf.plot(ax=ax, color='red', edgecolor='black', alpha=0.5)

                # Customize the plot
                legend_elements = [Patch(facecolor='red', edgecolor='black', alpha=0.5, label='Shapefile')]
                ax.legend(handles=legend_elements, loc='upper right')
                ax.set_title('Shapefile on Top of Image')

                # Show the plot
                plt.show()

        


                ###########################################################################################################

            except Exception as e:
                print(print_time()+"Raster_Process :: !!!!!!!!!! RASTER ERROR !!!!!!!!!!")
                print(print_time() + str(e))
                Error_Limit = Error_Limit - 1
                if Error_Limit < 1 :
                    print(print_time()+"Raster_Process :: !!!!!!!!!! RASTER ERROR 2 Time MoveFile to Image_Missin")
                    Move_File(Mid_name + "B03.jp2", Image + Track, Image_Missing + Track)
                    Move_File(Mid_name + "B04.jp2", Image + Track, Image_Missing + Track)
                    Move_File(Mid_name + "B08.jp2", Image + Track, Image_Missing + Track)
                    Move_File(Mid_name + "B12.jp2", Image + Track, Image_Missing + Track)
                    Move_File(Mid_name + "B1210.jp2", Image + Track, Image_Missing + Track)
                    Move_File(Mid_name + "B1210.jp2.aux.xml", Image + Track, Image_Missing + Track)
                    Move_File(Mid_name + "B1210.jp2.ovr", Image + Track, Image_Missing + Track)
                    Move_File(Mid_name + "B1210.jp2.xml", Image + Track, Image_Missing + Track)
                    Error_Limit = 2
                    
            ############################################## END Raster PROCESS################################################
        
            # ... 
        
        else:
            print(print_time() + f"Raster_Process :: {Full_name[:22]} Image not Found !!!!")
            print(BFB03, "_", os.path.exists(BFB03))
            print(BFB04, "_", os.path.exists(BFB04))
            print(BFB08, "_", os.path.exists(BFB08))
            print(BFB12, "_", os.path.exists(BFB12))
            print(AFB03, "_", os.path.exists(AFB03))
            print(AFB04, "_", os.path.exists(AFB04))
            print(AFB08, "_", os.path.exists(AFB08))
           
            # print("Joined Path:", BFB03)

            Move_File(f"{Mid_name}B03.jp2", os.path.join(Image, Track), os.path.join(Image_Missing, Track))
            Move_File(f"{Mid_name}B04.jp2", os.path.join(Image, Track), os.path.join(Image_Missing, Track))
            Move_File(f"{Mid_name}B08.jp2", os.path.join(Image, Track), os.path.join(Image_Missing, Track))
            Move_File(f"{Mid_name}B12.jp2", os.path.join(Image, Track), os.path.join(Image_Missing, Track))
            Move_File(f"{Mid_name}B1210.jp2", os.path.join(Image, Track), os.path.join(Image_Missing, Track))
            Move_File(f"{Mid_name}B1210.jp2.aux.xml", os.path.join(Image, Track), os.path.join(Image_Missing, Track))
            Move_File(f"{Mid_name}B1210.jp2.ovr", os.path.join(Image, Track), os.path.join(Image_Missing, Track))
            Move_File(f"{Mid_name}B1210.jp2.xml", os.path.join(Image, Track), os.path.join(Image_Missing, Track))
        
    print(print_time() + "Wait New Raster ::")
    mode = True

In [145]:
def main():
    current_dir = os.getcwd()

            # Print the current working directory
    print("Current Working Directory:", current_dir)
    global mode, Image_Post, Track_arr
    print(print_time() + "Application Start ::")
    while True:
        t = Timer(systemCooldown, loadCooldown)
        t.start()
        t.join()
        if mode:
            for Track in Track_arr:
                image_track = os.path.join(Image, Track)
                if os.path.exists(image_track):
                    rasters = [r for r in os.listdir(image_track) if r.endswith('B12.jp2')]
                    if rasters:
                        mode = False
                        print(print_time() + "Found NEW Raster :: ")
                        try:
                            Raster_Process(Track)
                        except Exception as e:
                            print(print_time() + "!!!!!!!!!SYSTEM ERROR !!!!!!!!!!")
                            print(print_time() + str(e))
                        print(print_time() + "Wait New Raster ::")
                else:
                    print(f"Directory not found: {image_track}")  # Inform if the directory does not exist

#Run the main function
main()

Current Working Directory: /Users/imdsk/RIDAProjectBurnScar
2024-03-03 16:32:48Application Start ::


2024-03-03 16:32:50Found NEW Raster :: 
/Users/imdsk/RIDAProjectBurnScar/Sentinel_Process/Image_Pre/T4/T47QNB_B12.jp2
2024-03-03 16:32:50Raster_Process :: T47QNB_20230330T034531 Image not Found !!!!
/Users/imdsk/RIDAProjectBurnScar/Sentinel_Process/Image_Pre/T4/T47QNB_B03.jp2 _ True
/Users/imdsk/RIDAProjectBurnScar/Sentinel_Process/Image_Pre/T4/T47QNB_B04.jp2 _ True
/Users/imdsk/RIDAProjectBurnScar/Sentinel_Process/Image_Pre/T4/T47QNB_B08.jp2 _ True
/Users/imdsk/RIDAProjectBurnScar/Sentinel_Process/Image_Pre/T4/T47QNB_B12.jp2 _ True
/Users/imdsk/RIDAProjectBurnScar/Sentinel_Process/Image/T4/T47QNB_20230330T034531_B03.jp2 _ True
/Users/imdsk/RIDAProjectBurnScar/Sentinel_Process/Image/T4/T47QNB_20230330T034531_B04.jp2 _ True
/Users/imdsk/RIDAProjectBurnScar/Sentinel_Process/Image/T4/T47QNB_20230330T034531_B08.jp2 _ True
2024-03-03 16:32:51Raster_Process :: Move File T47QNB_20230330T034531_B03.jp2 Complete
2024-03-03 16:32:52Raster_Process :: Move File T47QNB_20230330T034531_B04.jp2 Compl

KeyboardInterrupt: 